# Executive Summary
Global climate models are extremely valuable tools as they provide large scale estimates of climate data for wide-scale use. For this project, we were asked to use the ERA5 reanalysis temperature data to downscale precipitation data from the CPC local precipitation data. While global climate models offer climate data estimates for almost the entire world, they do have their drawbacks. One of these being the fact that it can be hard to use global climate models to model local-scale climate data. To help solve this issue that global climate models have, we can do what’s called downscaling to it to help offer higher resolution data at smaller scales. 

For this project, I developed a downscaling model that would build a model to downscale precipitation data from the temperature data through performing a principal component analysis and k-nearest neighbors model on the data. I also tried to run a random forest model on my data, but just could not get my code to work for it. I used principal component analysis to summarize the data in the large aggregated ERA5 Dataset to allow the data to be more easily analyzed. This data from the PCA was then inputted into a k-nearest neighbors model, which uses proximity to make predictions.

# Exploratory Data Analysis
## 1. Packages


In [ ]:
import Pkg; Pkg.add("MultivariateStats")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("NCDatasets")
import Pkg; Pkg.add("StatsBase")
import Pkg; Pkg.add("Unitful")
import Pkg; Pkg.add("DataFrames")
import Pkg; Pkg.add("DecisionTree")

In [ ]:
using Dates
using MultivariateStats
using Plots
using NCDatasets
using StatsBase
using Unitful
using DataFrames
using DecisionTree
Plots.default(; margin=4Plots.mm, size=(700, 400), linewidth=2)

## 2. Loading in Datasets
### 2.1. Precipitation Data
I first loaded in my precipitation dataset using NCDataset, and then made variables precip_time, precip_lon, precip_lat, and precip that represent the time, coordinates, and precipitation at a given coordinate at a given time between January 1, 2010 and December 31 of 2020.


In [ ]:
precip_ds = NCDataset("data/raw/precip_tx.nc") #loading in the precip_tx.nc dataset

In [ ]:
# Here, I'm creating variables from the precip_tx.nc dataset.
precip_time = precip_ds["time"][11324:15341] #saving the time data from 2010 through 2020
precip_time = Date.(precip_time)
precip_lon = precip_ds["lon"][:]
precip_lat = precip_ds["lat"][:]
precip_lat = reverse(precip_lat) #The latitude needs to be reversed.
precip = (precip_ds["precip"][:, :,11324:15341])*.1u"mm" #this precipitation data is in millimeters, so I'm adding in units.
precip = reverse(precip, dims=2) #Because we reversed the latitude, we need to do the same in the second dimension of the precipitation data.

In [ ]:
display(precip_ds[:precip])

In [ ]:
close(precip_ds) #closing the dataset

In [ ]:
heatmap(precip_lon, precip_lat, precip[:,:,1]';xlabel="Longitude", ylabel="Latitude", title="Preipitation on $(precip_time[1])")

### 2.2 Temperature Data

#### Loading in Temperature Data Year by Year
Next, I loaded in each temperature dataset from 2010 through 2020 individually, giving each dataset it's own time, lon, lat, and temp data.


In [ ]:
temp10_ds = NCDataset("data/raw/2m_temperature_2010.nc") 

In [ ]:
# Here, I'm creating variables from the 2m_temperature_2010.nc dataset.
temp10_time = temp10_ds["time"]
temp10_time = Date.(temp10_time)
temp10_lon = temp10_ds["longitude"][:]
temp10_lat = temp10_ds["latitude"][:]
temp10_lat = reverse(temp10_lat) #The latitude needs to be reversed.
temp10 = (temp10_ds["t2m"][:, :,:])*.1u"K" #this temp data is in Kelvin, so I'm adding in units.
temp10 = reverse(temp10, dims=2) #Because we reversed the latitude, we need to do the same in the second dimension of the temperature data.

In [ ]:
close(temp10_ds)

#### Repeating each step for temp datasets from 2011 to 2020


In [ ]:
temp11_ds = NCDataset("data/raw/2m_temperature_2011.nc") 

In [ ]:
temp11_time = temp11_ds["time"]
temp11_time = Date.(temp11_time)
temp11_lon = temp11_ds["longitude"][:]
temp11_lat = temp11_ds["latitude"][:]
temp11_lat = reverse(temp11_lat) 
temp11 = (temp11_ds["t2m"][:, :,:])*.1u"K" 
temp11 = reverse(temp11, dims=2)

In [ ]:
close(temp11_ds)

In [ ]:
temp12_ds = NCDataset("data/raw/2m_temperature_2012.nc") 

In [ ]:
temp12_time = temp12_ds["time"]
temp12_time = Date.(temp12_time)
temp12_lon = temp12_ds["longitude"][:]
temp12_lat = temp12_ds["latitude"][:]
temp12_lat = reverse(temp12_lat) 
temp12 = (temp12_ds["t2m"][:, :,:])*.1u"K" 
temp12 = reverse(temp12, dims=2) 

In [ ]:
close(temp12_ds)

In [ ]:
temp13_ds = NCDataset("data/raw/2m_temperature_2013.nc") 

In [ ]:
temp13_time = temp13_ds["time"]
temp13_time = Date.(temp13_time)
temp13_lon = temp13_ds["longitude"][:]
temp13_lat = temp13_ds["latitude"][:]
temp13_lat = reverse(temp13_lat)
temp13 = (temp13_ds["t2m"][:, :,:])*.1u"K" 
temp13 = reverse(temp13, dims=2)

In [ ]:
close(temp13_ds)

In [ ]:
temp14_ds = NCDataset("data/raw/2m_temperature_2014.nc") 

In [ ]:
temp14_time = temp14_ds["time"]
temp14_time = Date.(temp14_time)
temp14_lon = temp14_ds["longitude"][:]
temp14_lat = temp14_ds["latitude"][:]
temp14_lat = reverse(temp14_lat) 
temp14 = (temp14_ds["t2m"][:, :,:])*.1u"K" 
temp14 = reverse(temp14, dims=2) 

In [ ]:
close(temp14_ds)

In [ ]:
temp15_ds = NCDataset("data/raw/2m_temperature_2015.nc") 

In [ ]:
temp15_time = temp15_ds["time"]
temp15_time = Date.(temp15_time)
temp15_lon = temp15_ds["longitude"][:]
temp15_lat = temp15_ds["latitude"][:]
temp15_lat = reverse(temp15_lat) 
temp15 = (temp15_ds["t2m"][:, :,:])*.1u"K" 
temp15 = reverse(temp15, dims=2) 

In [ ]:
close(temp15_ds)

In [ ]:
temp16_ds = NCDataset("data/raw/2m_temperature_2016.nc") 

In [ ]:
temp16_time = temp16_ds["time"]
temp16_time = Date.(temp16_time)
temp16_lon = temp16_ds["longitude"][:]
temp16_lat = temp16_ds["latitude"][:]
temp16_lat = reverse(temp16_lat) 
temp16 = (temp16_ds["t2m"][:, :,:])*.1u"K" 
temp16 = reverse(temp16, dims=2) 

In [ ]:
close(temp16_ds)

In [ ]:
temp17_ds = NCDataset("data/raw/2m_temperature_2017.nc") 

In [ ]:
temp17_time = temp17_ds["time"]
temp17_time = Date.(temp17_time)
temp17_lon = temp17_ds["longitude"][:]
temp17_lat = temp17_ds["latitude"][:]
temp17_lat = reverse(temp17_lat) 
temp17 = (temp17_ds["t2m"][:, :,:])*.1u"K" 
temp17 = reverse(temp17, dims=2)

In [ ]:
close(temp17_ds)

In [ ]:
temp18_ds = NCDataset("data/raw/2m_temperature_2018.nc") 

In [ ]:
temp18_time = temp18_ds["time"]
temp18_time = Date.(temp18_time)
temp18_lon = temp18_ds["longitude"][:]
temp18_lat = temp18_ds["latitude"][:]
temp18_lat = reverse(temp18_lat) 
temp18 = (temp18_ds["t2m"][:, :,:])*.1u"K" 
temp18 = reverse(temp18, dims=2)

In [ ]:
close(temp18_ds)

In [ ]:
temp19_ds = NCDataset("data/raw/2m_temperature_2019.nc") 

In [ ]:
temp19_time = temp19_ds["time"]
temp19_time = Date.(temp19_time)
temp19_lon = temp19_ds["longitude"][:]
temp19_lat = temp19_ds["latitude"][:]
temp19_lat = reverse(temp19_lat)
temp19 = (temp19_ds["t2m"][:, :,:])*.1u"K" 
temp19 = reverse(temp19, dims=2)

In [ ]:
close(temp19_ds)

In [ ]:
temp20_ds = NCDataset("data/raw/2m_temperature_2020.nc") 

In [ ]:
temp20_time = temp20_ds["time"]
temp20_time = Date.(temp20_time)
temp20_lon = temp20_ds["longitude"][:]
temp20_lat = temp20_ds["latitude"][:]
temp20_lat = reverse(temp20_lat) 
temp20 = (temp20_ds["t2m"][:, :,:])*.1u"K" 
temp20 = reverse(temp20, dims=2)

In [ ]:
close(temp20_ds)

### 2.3 Creating One Large Dataset That Combines all Temp Dat & Converting Times to Daily Instead of Hourly
After loading in the temperature data from each year from 2010 through 2020, I then aggregated all of the time and temperature data into 2 large variables called temp and temp_time.


In [ ]:
# Function to read temperature data for a given year
function read_temperature_data(year)
    ds = NCDataset("data/raw/2m_temperature_$(year).nc")
    
    time = Date.(ds["time"])
    lon = ds["longitude"][:]
    lat = reverse(ds["latitude"][:])
    temp = (ds["t2m"][:, :, :]) * 0.1u"K"
    temp = reverse(temp, dims=2)
    
    close(ds)
    
    return time, lon, lat, temp
end

# Function to aggregate hourly data to daily data
function aggregate_to_daily(time, temp)
    unique_dates = unique(time)
    daily_temp = []

    for date in unique_dates
        indices = findall(x -> x == date, time)
        daily_mean = mean(temp[:, :, indices], dims=3)
        push!(daily_temp, daily_mean)
    end

    return unique_dates, cat(daily_temp..., dims=3)
end

# Years to process
years = 2010:2020

# Initialize arrays to store aggregated data
all_aggregated_times = []
all_aggregated_temps = []

# Loop over years and read/aggregate data
for year in years
    time, _, _, temp = read_temperature_data(year)
    aggregated_times, aggregated_temp = aggregate_to_daily(time, temp)
    
    push!(all_aggregated_times, aggregated_times)
    push!(all_aggregated_temps, aggregated_temp)
end

# Combine aggregated data
combined_aggregated_times = cat(all_aggregated_times..., dims=1)
combined_aggregated_temps = cat(all_aggregated_temps..., dims=3)

In [ ]:
temp_lat = temp10_lat[1:14] #All of the lat data from the different temp datasets are the same, so I represent the lat for the whole temp data just as one of lat variables I made earlier
temp_lon = temp10_lon[30:41] #All of the lon data from the different temp datasets are the same, so I represent the lon for the whole temp data just as one of lon variables I made earlier
temp_time = combined_aggregated_times
temp = combined_aggregated_temps[30:41,1:14,:]

:::{.callout-note}
I'm also matching the latitude and longitude ranges of the temperature data to match with the precipitation data here. 
:::


In [ ]:
@assert temp_time == precip_time #Making sure that my two time variables are the same

In [ ]:
heatmap(temp_lon, temp_lat, temp[:,:,1]'; xlabel="Longitude", ylabel="Latitude", title="Temperature on $(temp_time[1])")

## 3. Splitting Data into Testing & Training data
I then split my data into test and training data, with 70% being training data, and 30% being test data.


In [ ]:
test_start_date = Date(2017, 09, 13) #Splitting the test and training data 70%-30%
test_start_index = searchsortedfirst(temp_time, test_start_date)

In [ ]:
# Here I'm splitting my data into training & testing sets

precip_train = precip[:, :, 1:test_start_index - 1]
precip_test = precip[:, :, test_start_index:end]

temp_train = temp[:, :, 1:test_start_index - 1]
temp_test = temp[:, :, test_start_index:end]

time_train = temp_time[1:test_start_index - 1]
time_test = temp_time[test_start_index:end]

## 4. Preprocessing
I then preprocessed my temperature data to get information about the variance and reshape it for use in PCA analysis.


In [ ]:
# Preprocessing the data here to get data about the variance and reshape the variance data so it can be used for PCA
function preprocess(temp::Array{T,3}, temp_reference::Array{T,3})::AbstractMatrix where {T}
    n_lon, n_lat, n_time = size(temp)
    climatology = mean(temp_reference; dims=3)
    temp_anom = temp .- climatology

    #reshape to 2D
    temp_anom = reshape(temp_anom, n_lon * n_lat, n_time)

    #stripping units
    return ustrip.(u"K", temp_anom)
end

In [ ]:
n_lon, n_lat, n_time = size(temp)

temp_mat_train = preprocess(temp_train, temp_train)
temp_mat_test = preprocess(temp_test, temp_train)

# Methods 

## 1. Principal Components
I then ran principal components analysis on the temperature training data through fitting the training data to PCA and plotting the variance and cumulative variance.
### 1.1 Fitting


In [ ]:
pca_model = fit(PCA, temp_mat_train; maxoutdim=10, pratio=0.99);
pca_model

In [ ]:
p1 = plot(principalvars(pca_model)/var(pca_model); xlabel="# of PC's", xticks= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], ylabel= "Fraction of Variance Explained", label = false, title= "Variance Explained")

p2 = plot(cumsum(principalvars(pca_model))/var(pca_model); xlabel="# of PC's", xticks= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], ylabel= "Fraction of Variance Explained", label = false, title= "Cumulative Variance Explained")

plot(p1,p2; layout=(1,2), size=(900,400))

After looking at the cumulative variance explained plot, I decided to use 3 principal components because those three account for almost 0.95 of the cumulative variance, but reduces alot of the noise.


In [ ]:
p = []
for i in 1:3
    pc = projection(pca_model)[:, i]
    pc_reshape = reshape(pc, n_lon, n_lat)'
    pi = heatmap(
        temp_lon,
        temp_lat,
        pc_reshape;
        xlabel="Longitude",
        ylabel="Latitude",
        title="PC $i",
        aspect_ratio=:equal,
        cmap=:PuOr
    )
    push!(p, pi)
end
plot(p...; layout=(1, 3), size=(1800, 600))

In [ ]:
pc_ts = StatsBase.predict(pca_model, temp_mat_train)
day_of_year = Dates.dayofyear.(temp_time)

p = []
for i in 1:3
    pi = scatter(
        day_of_year,
        pc_ts[i, :];
        xlabel="Day of Year",
        ylabel="PC $i",
        title="PC $i",
        label=false,
        alpha=0.3,
        color=:gray
    )
    push!(p, pi)
end
plot(p...; layout=(1, 3), size=(1800, 600))

From this plot, we see that principal components 1 & 2 appear to have some sort of relation to the difference in temperatures on land and in the ocean, while the third prinicipal component appears to have some sort of relation to longitude.


In [ ]:
Months = Dates.month.(temp_time)

p=[]
for i in 1:3
    pi=scatter(
        Months,
        pc_ts[i, :];
        xlabel= "Months in a Year",
        ylabel = "PC $i",
        title = "PC $i",
        label = false,
        alpha= 0.3,
        color=:blue,
        xticks =[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    )
    push!(p, pi)
end
plot(p...; layout=(1,3), size=(1800, 600))    

From this plot, we can see that principal component 1 is definitely driving the seasonal cycle of the data, principal components 2 and 3 appear more closesly related with daily variances in temperature.


In [ ]:
avg_precip = ustrip.(u"mm", [mean(skipmissing(precip_train[:, :, t])) for t in 1:size(precip_train, 3)])
avg_precip = replace(avg_precip, NaN => 0)

In [ ]:
p1 = scatter(
    pc_ts[2, :],
    pc_ts[3, :];
    zcolor=avg_precip,
    xlabel="PC 2",
    ylabel="PC 3",
    markersize=3,
    clims=(0, 2.75),
    title="All Days",
    label=false
)

p2_idx = findall(avg_precip .> quantile(avg_precip, 0.98))
p2 = scatter(
    pc_ts[2, p2_idx],
    pc_ts[3, p2_idx];
    zcolor=avg_precip[p2_idx],
    xlabel="PC 2",
    ylabel="PC 3",
    markersize=5,
    clims=(0, 2.75),
    title="Rainy Days",
    label=false
)
plot(p1, p2; size=(1800, 600), link=:both)

## 6. K-NN Model


In [ ]:
function euclidean_distance(x::AbstractVector, y::AbstractVector)::AbstractFloat
    return sqrt(sum((x .- y) .^ 2))
end
function nsmallest(x::AbstractVector, n::Int)::Vector{Int}
    idx = sortperm(x)
    return idx[1:n]
end
function knn(X::AbstractMatrix, X_i::AbstractVector, K::Int)::Tuple{Int,AbstractVector}
    # calculate the distances between X_i and each row of X
    dist = [euclidean_distance(X_i, X[j, :]) for j in 1:size(X, 1)]
    idx = nsmallest(dist, K)
    w = 1 ./ dist[idx]
    w ./= sum(w)
    idx_sample = sample(idx, Weights(w))
    return (idx_sample, vec(X[idx_sample, :]))
end

In [ ]:
function predict_knn(temp_train, temp_test, precip_train; n_pca::Int)
    X_train = preprocess(temp_train, temp_train)
    X_test = preprocess(temp_test, temp_train)

    # fit the PCA model to the training data
    pca_model = fit(PCA, X_train; maxoutdim=n_pca)

    # project the test data onto the PCA basis
    train_embedded = StatsBase.predict(pca_model, X_train)
    test_embedded = StatsBase.predict(pca_model, X_test)

    # use the `knn` function for each point in the test data
    precip_pred = map(1:size(X_test, 2)) do i
        idx, _ = knn(train_embedded', test_embedded[:, i], 3)
        precip_train[:, :, idx]
    end
# return a matrix of predictions
    return precip_pred
end

In [ ]:
t_sample = rand(1:size(temp_test, 3), 3)
precip_pred = predict_knn(temp_train, temp_test[:, :, t_sample], precip_train; n_pca=3)

In [ ]:
p = map(eachindex(t_sample)) do ti
    t = t_sample[ti]
    y_pred = precip_pred[ti]'
    y_actual = precip_test[:, :, t]'
    cmax = max(maximum(skipmissing(y_pred)), maximum(skipmissing(y_actual)))
    cmax = ustrip(u"mm", cmax)

    p1 = heatmap(
        precip_lon,
        precip_lat,
        y_pred;
        xlabel="Longitude",
        ylabel="Latitude",
        title="Predicted",
        aspect_ratio=:equal,
        clims=(0, cmax)
    )

    p2 = heatmap(
        precip_lon,
        precip_lat,
        y_actual;
        xlabel="Longitude",
        ylabel="Latitude",
        title="Actual",
        aspect_ratio=:equal,
        clims=(0, cmax)
    )

    plot(p1, p2; layout=(2, 1), size=(1000, 400))
end
plot(p...; layout=(2, 3), size=(1500, 1200))

## 7. Random Forest Model


In [ ]:
function train_random_forest(temp_train, temp_test::Any, precip_train::Any; n_pca::Int, n_trees::Int)
    X_train = preprocess(temp_train, temp_train)
    X_test = preprocess(temp_test, temp_train)
    
    PCA_model = fit(PCA, X_train, maxoutdim=n_pca)

    y_train = avg_precip

    # Flatten to 2D arrays
    train_embedded = StatsBase.predict(PCA_model, X_train)
    test_embedded = StatsBase.predict(PCA_model, X_test)

    y_train_no_missing = avg_precip[.!ismissing.(avg_precip)]


    # Train the Random Forest model
    rf_model = DecisionTree.build_forest(
        y_train_no_missing,
        train_embedded[:, :, findall(!ismissing, y_train)],
        n_trees
    )
    
    return rf_model
end

In [ ]:
rf_model = train_random_forest(temp_train, temp_test, precip_train; n_pca=3, n_trees=100)

# Model Comparison
I tried to create a random forest model in order to make a comparison about which model worked best for my data, but couldn't get my code for the random forest model to work. However, I do believe that my random forest model, if using the original data and not the PCA data, could have been better for my large dataset. I think a big reason my KNN model didn't do great was because there was still some noise from outliers even after choosing my principal components. Because KNN models are sensitive to noise, that noise may have caused my model to struggle.

# Conclusion
I think that overall, while my KNN model didn't work great and I couldn't get a second model to run, this project wasn't a complete failure. I think that even though I didn't get the results I wanted, I ended up deepening my understanding of PCA, KNN models, and random forest models hrough the process of trying to put this project together.